<a href="https://colab.research.google.com/github/Alexandre-Data/OC-Projet-7/blob/main/Projet7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!git version
!pip install pyngrok
!pip install mlflow
!pip install SMOTE
!pip install -U imbalanced-learn
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from pyngrok import ngrok
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold

from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
import smote
import mlflow

from sklearn.linear_model import LogisticRegression


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
git version 2.25.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
df=pd.read_csv("/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/top_95_features.csv")

In [3]:
df.head()

,index,SK_ID_CURR,TARGET,EXT_SOURCE_2,EXT_SOURCE_3,index.1,DAYS_BIRTH,DAYS_REGISTRATION,SK_ID_CURR.1,DAYS_ID_PUBLISH,...,REFUSED_RATE_DOWN_PAYMENT_MIN,BURO_MONTHS_BALANCE_MAX_MAX,CC_SK_DPD_SUM,CC_SK_DPD_DEF_VAR,ACTIVE_MONTHS_BALANCE_MAX_MAX,BURO_STATUS_5_MEAN_MEAN,WEEKDAY_APPR_PROCESS_START_SATURDAY,BURO_CREDIT_TYPE_Car loan_MEAN,ORGANIZATION_TYPE_Construction,NAME_FAMILY_STATUS_Separated
0,0,100002,1.0,0.262949,0.139376,0,-9461,-3648.0,100002,-2120,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
1,1,100003,0.0,0.622246,0.000000,1,-16765,-1186.0,100003,-291,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
2,2,100004,0.0,0.555912,0.729567,2,-19046,-4260.0,100004,-2531,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
3,3,100006,0.0,0.650442,0.000000,3,-19005,-9833.0,100006,-2437,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
4,4,100007,0.0,0.322738,0.000000,4,-19932,-4311.0,100007,-3458,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0


In [4]:
non_finite_values = np.isfinite(df)
df = df.replace([np.inf, -np.inf], np.nan).dropna()


In [5]:
#CODE SANS SMOTE
X = df.drop('TARGET', axis=1)
y = df['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
dt = DecisionTreeClassifier(random_state=42)

# On fit
dt.fit(X_train, y_train)

# On prédit
y_pred = dt.predict(X_test)

In [6]:
# Calculer la matrice de confusion
confusion_mat = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_mat)

Matrice de confusion :
[[61285  4996]
 [ 4086   880]]


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Calculer différents scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Afficher les scores
print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Precision: {precision*100:.2f}%')
print(f'Recall: {recall*100:.2f}%')
print(f'F1-Score: {f1*100:.2f}%')
print(f'ROC AUC Score: {roc_auc*100:.2f}%')


Accuracy: 87.25%
Precision: 14.98%
Recall: 17.72%
F1-Score: 16.23%
ROC AUC Score: 55.09%


In [8]:
#CODE AVEC SMOTE
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Créez une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Ajustez le modèle à l'ensemble d'entraînement suréchantillonné
dt.fit(X_train_resampled, y_train_resampled)

# Faites des prédictions sur l'ensemble de test
y_pred = dt.predict(X_test)

Accuracy: 84.79%


In [9]:
# Calculer la matrice de confusion
confusion_mat = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_mat)

Matrice de confusion :
[[59426  6855]
 [ 3983   983]]


In [10]:
# Calculer différents scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Afficher les scores
print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Precision: {precision*100:.2f}%')
print(f'Recall: {recall*100:.2f}%')
print(f'F1-Score: {f1*100:.2f}%')
print(f'ROC AUC Score: {roc_auc*100:.2f}%')


Accuracy: 84.79%
Precision: 12.54%
Recall: 19.79%
F1-Score: 15.35%
ROC AUC Score: 54.73%


In [ ]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Créer une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(dt, param_grid, scoring='accuracy', cv=5)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)


In [ ]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Créer une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(dt, param_grid, scoring='accuracy', cv=5)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

In [ ]:
df1=pd.read_csv("/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/top_80_features.csv")

In [ ]:
X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Créer une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(dt, param_grid, scoring='accuracy', cv=5)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

make_scorer utoliser ça pour créer mon score à moi . augmenter *10 les erreurs de type 1 et 2 dans la matrice de confusion. Utiliser un dummy classifier une fois mon modele fait, puis le faire tourner sur un algo optimisé.Graphiques: mettre features importance; mettre une jauge, faire des histogrammes. dashboard pour le client, facile à comprendre. un truc non opti pour neophyte, et un truc opti pour dire pourquoi.